***

*Course:* [Math 535](https://people.math.wisc.edu/~roch/mmids/) - Mathematical Methods in Data Science (MMiDS)  
*Chapter:* 5-Random walks on graphs and Markov chains  
*Author:* [Sebastien Roch](https://people.math.wisc.edu/~roch/), Department of Mathematics, University of Wisconsin-Madison  
*Updated:* Jan 7, 2024   
*Copyright:* &copy; 2024 Sebastien Roch

***

In [ ]:
# IF RUNNING ON GOOGLE COLAB, UNCOMMENT THE FOLLOWING CODE CELL
# When prompted, upload: 
#     * mmids.py
#     * mathworld-adjacency.csv
#     * mathworld-titles.csv
# from your local file system
# Files at: https://github.com/MMiDS-textbook/MMiDS-textbook.github.io/tree/main/utils
# Alternative instructions: https://colab.research.google.com/notebooks/io.ipynb

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
# PYTHON 3
import numpy as np
from numpy import linalg as LA
from numpy.random import default_rng
rng = default_rng(535)
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import tensorflow as tf
from tensorflow import keras
import mmids

## Motivating example: ranking webpages

A common task in network analysis is to identify "central" vertices in a graph. Centrality is a vague concept. It can be defined in many different ways depending on the context and the type of network. Quoting from [Wikipedia](https://en.wikipedia.org/wiki/Centrality):

> In graph theory and network analysis, indicators of centrality assign numbers or rankings to nodes within a graph corresponding to their network position. Applications include identifying the most influential person(s) in a social network, key infrastructure nodes in the Internet or urban networks, super-spreaders of disease, and brain networks. [...] Centrality indices are answers to the question "What characterizes an important vertex?" The answer is given in terms of a real-valued function on the vertices of a graph, where the values produced are expected to provide a ranking which identifies the most important nodes. The word "importance" has a wide number of meanings, leading to many different definitions of centrality. 

In an undirected graph, a natural approach is to look at the degree of a vertex as a measure of its importance (also referred to as degree centrality). But it is hardly the only one. One could for instance look at the average distance to all other nodes (its reciprocal is the [closeness centrality](https://en.wikipedia.org/wiki/Closeness_centrality)) or at the number of shortest paths between pairs of vertices going through the vertex (known as [betweenness centrality](https://en.wikipedia.org/wiki/Betweenness_centrality)). 

What if the graph is directed? Things are somewhat more complicated there. For instance, there is now the in-degree as well as the out-degree. 

Let us look at a particular example of practical importance, the World Wide Web (from now on, the Web). In this case, the vertices are webpages and a directed edge from $u$ to $v$ indicates a hyperlink from page $u$ to page $v$. The Web is much too large to analyze here. Instead, we will consider a tiny (but still interesting!) subset of it, the pages of [Wolfram's MathWorld](https://mathworld.wolfram.com), a wonderful mathematics resource. 

Each page of MathWorld concerns a particular mathematical concept, e.g., [scale-free network](https://mathworld.wolfram.com/Scale-FreeNetwork.html). A definition and notable properties are described. Importantly for us, in a section entitled "SEE ALSO", other related mathematical concepts are listed with a link to their MathWorld page. In the case of scale-free networks, the [small world network](https://mathworld.wolfram.com/SmallWorldNetwork.html) topic is referenced, among others.

The resulting directed graph is available through the [NetSet](https://netset.telecom-paris.fr/index.html) datasets and can be downloaded [here](https://netset.telecom-paris.fr/pages/mathworld.html). We load it now. For convenience, we have reformatted it into the files `mathworld-adjacency.csv` and `mathworld-titles.csv`, which are available on the [GitHub of the book](https://github.com/MMiDS-textbook/MMiDS-textbook.github.io/tree/main/utils/datasets).

In [ ]:
df_edges = pd.read_csv('mathworld-adjacency.csv')
df_edges.head()

It consists in a list of directed edges. For example, the first one is an edge from vertex `0` to vertex `2`. The second one is from `1` to `47` and so on. 

There is a total of $49069$ edges.

In [ ]:
df_edges.shape[0]

The second file contains the titles of the pages.

In [ ]:
df_titles = pd.read_csv('mathworld-titles.csv')
df_titles.head()

So the first edge above is from `Alexander's Horned Sphere` to `Antoine's Horned Sphere`. That is, the [latter](https://mathworld.wolfram.com/AntoinesHornedSphere.html) is listed in the SEE ALSO section of the [former](https://mathworld.wolfram.com/AlexandersHornedSphere.html). 

There are $12362$ topics.

In [ ]:
df_titles.shape[0]

We construct the graph by adding the edges one by one. We first convert `df_edges` into a Numpy array.

In [ ]:
edgelist = df_edges[['from','to']].to_numpy()
print(edgelist)

In [ ]:
n = 12362
G = nx.empty_graph(n, create_using=nx.DiGraph)
for i in range(edgelist.shape[0]):
    G.add_edge(edgelist[i,0], edgelist[i,1])

In [ ]:
G.in_degree(0)

while that of `Antoine's Horned Sphere` is:

In [ ]:
G.in_degree(2)

suggesting that the former is more central than the latter, at least in the sense that it is referenced more often.

But is that the right measure? Consider the following: `Antoine's Horned Sphere` receives only one reference, but it is from a seemingly relatively important vertex, `Alexander's Horned Sphere`. How can one take this into account in quantifying its importance in the network?

We will come back to this question later in this chapter. To hint at things to come, it will turn out that "exploring the graph at random" provides a powerful perspective on centrality.  